## Imports

In [104]:
import pyxdf 
import numpy as np
import pandas as pd
from lmfit.models import Model
from os import listdir, getcwd
from os.path import isfile, join
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pywt
import math
from pandas.api.types import CategoricalDtype

## Pupillary Functions


Task evoked pupillary response is calculated after correcting for luminance-induced pupil dilation: $𝑇𝐸𝑃𝑅 = 𝑑_m − 𝑑(𝑌)$, where $d_m$ is the measured pupil dilation, and $d(Y)$ is the predicted pupil dilation for the given luminance level. 

Predicted pupil dilation is calculated from a calibration sequence that produces and individual mapping model for each participant. The calibration sequence consists of 8 solid gray colors with varying luminance levels displayed in a psuedo-random order for 6 seconds each. The luminance levels span the range from 0.0 to 0.78, and for each calibration level, the first 0.5s of data is discarded to account for the initial pupillary response to the change in luminance, which can take a maximum of 0.5s. . The individual mapping model is calculated using a non-linear least squares regression to fit the equation $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$ to the measured pupil dilation data for each participant. 

Pupil dilation data and the average luminance data were collected at 90 Hz, the display rate of the HMD.

See: Eckert, M., Robotham, T., Habets, E. A. P., and Rummukainen, O. S. (2022). Pupillary Light Reflex Correction for Robust Pupillometry in Virtual Reality. Proc. ACM Comput. Graph. Interact. Tech. 5, 1–16. doi: 10.1145/3530798

In [2]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [3]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours , and strictly larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [4]:
def ipa_func(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym16', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]
    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt. threshold (cD2m ,λuniv,mode="hard")
    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA

## Data Processing Functions

In [5]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.round(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)
    return dfs

In [6]:
accom_time = pd.to_timedelta(0.5, unit='s')

In [7]:
method_cats = CategoricalDtype(['4DoF','6DoF', 'unimanual','bimanual'], ordered=False)
model_cats = CategoricalDtype(['A', 'B', 'C', 'D'], ordered=True)
event_cats = CategoricalDtype(['Start', 'PointPlaced', 'Move', 'End', 'Draw', 'Erase', 'PointDeleted'])
data_names = ['id', 'model', 'method']

In [8]:
def process_gaze_luminance_data(stream_df):
    pupil = stream_df['GazeStream'].loc[(stream_df['GazeStream']['LeftEyeIsBlinking'] == 0) 
                                        & (stream_df['GazeStream']['RightEyeIsBlinking'] == 0) 
                                        & (stream_df['GazeStream']['LeftPupilDiameter'] > 0) 
                                        & (stream_df['GazeStream']['RightPupilDiameter'] > 0), 
                                        ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
    pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

    lum = stream_df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
    lum['time'] = pd.to_timedelta(lum['time'], unit='s')

    # Intersection of time stamps
    pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

    # Filter pupil and luminance data by intersection
    pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
    lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

    # Combined DataFrame for pupil and luminance
    pupil_lum = pd.DataFrame({
        'time': pd.to_timedelta(pupil_lum_time_intersection, unit='s'),
        'luminance': lum['Luminance'],
        'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
        'methodID': pupil['MethodID'],
        'modelID': pupil['ModelID']
    })

    return pupil_lum

In [9]:
def process_calibration_data(pupil_lum_df, stream_df):
    calibration_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | 
                                                           (stream_df['ExperimentStream']['SceneEvent'] == 'Calibration') | 
                                                           (stream_df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), 
                                                           ['time','SceneEvent', 'EventType']]
    calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
    c_start_times = calibration_events[:8]['time']
    c_end_times = calibration_events[1:]['time']
    c_start_times.reset_index(drop=True, inplace=True)
    c_end_times.reset_index(drop=True, inplace=True)

    calib_data = {}
    for i in range(8):
        calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
        calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
        calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

    calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)
    return calibration_data

In [10]:
def process_navigation_data(pupil_lum_df, stream_df, a, b, c):
    grouped_data = stream_df['NavigationStream'].groupby(['ModelID', 'MethodID'])

    stream_df['SurveyStream']['ModelID'] = stream_df['SurveyStream']['ModelID'].astype(float)
    stream_df['SurveyStream']['MethodID'] = stream_df['SurveyStream']['MethodID'].astype(float)
    
    discomfort_survey = stream_df['SurveyStream'].loc[
        (stream_df['SurveyStream']['SurveyType'] == 'Discomfort') & 
        (stream_df['SurveyStream']['ModelID'] < 4), 
        ['time', 'ModelID', 'MethodID']]
    survey_group = discomfort_survey.groupby(['ModelID', 'MethodID'])

    start_times = []
    end_times = []

    for i in range(4):
        for j in range(2,4):
            trial = grouped_data.get_group((i, j))

            start = trial.loc[(trial['spline_percent'] > 0.001)].index[0]
            start_time = pd.to_timedelta(stream_df['NavigationStream'].loc[start, 'time'], unit='s')

            end = trial.loc[(trial['spline_percent'] > 0.995)]
            end_time = 0
            # For 6DoF navigation, completion was determined by collision with bounding box
            # Spline percentage was based on projection, so it may not reach > 0.995.
            # In this case, the survey time serves as the end time (rather than lowering the threshold)
            if len(end) > 0:
                end = end.index[0]
                end_time = pd.to_timedelta(stream_df['NavigationStream'].loc[end, 'time'], unit='s')
            else:
                end = survey_group.get_group((i, j)).index[0]
                end_time = pd.to_timedelta(stream_df['SurveyStream'].loc[end, 'time'], unit='s') - pd.offsets.Second(3)
            
            start_times.append(start_time)
            end_times.append(end_time)


    nav_start_times = start_times
    nav_end_times = end_times

    nav_data = {}
    for i in range(8):
        nav_data[i] = pupil_lum_df.loc[
            (pupil_lum_df['luminance'] >0) & 
            (pupil_lum_df['time']>nav_start_times[i]) & 
            (pupil_lum_df['time']<nav_end_times[i]), 
            ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        nav_data[i].reset_index(drop=True, inplace=True)

    navigation_data = pd.concat(nav_data, names=['trial'])
    navigation_data['pupil_lum_base'] = pupil_func(navigation_data['luminance'], a, b, c)
    navigation_data['adj_pupil'] = navigation_data['pupilDiameter'] - navigation_data['pupil_lum_base']

    return navigation_data

In [11]:
def process_creation_data(pupil_lum_df, stream_df, a, b, c):
    
    crt_start_times = stream_df['CreationStream'].loc[
    (stream_df['CreationStream']['EventName'] == 'StartPointRegistered'), 
    ['time', 'ModelID', 'MethodID']]
    crt_start_times = pd.to_timedelta(crt_start_times.groupby(['ModelID', 'MethodID']).first()['time'], unit='s') + pd.offsets.Second(2)
    crt_start_times.reset_index(drop=True, inplace=True)

    crt_end_times = stream_df['CreationStream'].loc[
    (stream_df['CreationStream']['EventName'] == 'FinishPath'), 
    ['time', 'ModelID', 'MethodID']]
    crt_end_times = pd.to_timedelta(crt_end_times.groupby(['ModelID', 'MethodID']).first()['time'], unit='s')
    crt_end_times.reset_index(drop=True, inplace=True)
    
    crt_data = {}
    for i in range(8):
        crt_data[i] = pupil_lum_df.loc[
            (pupil_lum_df['time'] > crt_start_times.loc[i]) & 
            (pupil_lum_df['time'] < crt_end_times.loc[i]), 
            ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        crt_data[i].reset_index(drop=True, inplace=True)

    creation_data = pd.concat(crt_data, names=['trial'])
    creation_data['pupil_lum_base'] = pupil_func(creation_data['luminance'], a, b, c)
    creation_data['adj_pupil'] = creation_data['pupilDiameter'] - creation_data['pupil_lum_base']
    
    return creation_data

In [12]:
def process_creation_stats(stream_df):
    
    group = stream_df['CreationStream'].groupby(['ModelID', 'MethodID'])
    creation_counts  = []

    for i in range(4):
            for j in range(0,2):
                trial = group.get_group((i,j))
                creation_counts.append(trial.groupby('EventType', observed=False).size().fillna(0))
    
    keys = [(i,j) for i in range(4) for j in range(0,2)]
    creation_stats = pd.concat(creation_counts, axis=0, keys=keys, names=['ModelID', 'MethodID']).unstack(level=2)
    creation_stats = creation_stats.drop(columns=['End', 'Start'])
    return creation_stats

In [13]:
def process_discomfort_data(stream_df):
    discomfort_values = stream_df['SurveyStream'].loc[stream_df['SurveyStream']['SurveyType'] == 'Discomfort', ['time', 'Value', 'ModelID', 'MethodID']]
    discomfort_values['time'] = pd.to_timedelta(discomfort_values['time'], unit='s')
    discomfort_values.reset_index(drop=True, inplace=True)
    return discomfort_values

In [14]:
def process_seq_data(stream_df):
    seq_values = stream_df['SurveyStream'].loc[stream_df['SurveyStream']['SurveyType'] == 'SEQ', ['time', 'Value', 'ModelID', 'MethodID']]
    seq_values['time'] = pd.to_timedelta(seq_values['time'], unit='s')
    seq_values.reset_index(drop=True, inplace=True)
    return seq_values

In [15]:
def process_ipa_calc(data):
    methods = []
    models = []
    ipa = []
    for i in range(8):
        methods.append(data.loc[i]['methodID'].iloc[i])
        models.append(data.loc[i]['modelID'].iloc[i])
        pupil = data.loc[i]['pupilDiameter']
        pupil.index = data.loc[i]['time']
        ipa.append(ipa_func(pupil))
        
    return pd.DataFrame({'methodID': methods, 'modelID': models, 'IPA': ipa})

## Statistical Functions

In [227]:
def iqr_outlier_indices(data):
    q1 = data.quantile(.25)
    q3 = data.quantile(.75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return np.where((data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr)))

In [17]:
def iqr_stats(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return iqr, q1, q3

## Import Data

In [18]:
data_dir = join(getcwd(),'Path_Data')
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]

In [19]:
dfs = []
for file in data_files:
    dfs.append(import_data(file))

## Process Data

In [20]:
user_dfs_nav = []
user_dfs_crt = []
ids = []

for df in dfs:
    id = df['ExperimentStream']['UserID'][0]
    ids.append(id)

    # Remove final empty row from string data streams
    df['SurveyStream'] = df['SurveyStream'].replace(r'^\s*$', np.nan, regex=True).dropna()
    df['CreationStream'] = df['CreationStream'].replace(r'^\s*$', np.nan, regex=True).dropna()

    df['SurveyStream']['ModelID'] = df['SurveyStream']['ModelID'].astype(float)
    df['SurveyStream']['MethodID'] = df['SurveyStream']['MethodID'].astype(float)
    df['SurveyStream']['Value'] = df['SurveyStream']['Value'].astype(float)

    df['CreationStream']['ModelID'] = df['CreationStream']['ModelID'].astype(float)
    df['CreationStream']['MethodID'] = df['CreationStream']['MethodID'].astype(float)
    df['CreationStream']['EventType'] = df['CreationStream']['EventType'].astype(event_cats)

    pupil_lum_df = process_gaze_luminance_data(df)
    calibration_data = process_calibration_data(pupil_lum_df, df)

    # Fit pupil response to luminance
    x_data = calibration_data['luminance']
    y_data = calibration_data['pupilDiameter']
    exp_mod = Model(pupil_func)
    params = exp_mod.make_params(a=1, b=4, c=0)
    result = exp_mod.fit(y_data, params, x=x_data)
    a = result.params['a'].value
    b = result.params['b'].value
    c = result.params['c'].value

    navigation_data = process_navigation_data(pupil_lum_df, df, a, b, c)
    ipa_calc_nav = process_ipa_calc(navigation_data)
    creation_data = process_creation_data(pupil_lum_df, df, a, b, c)
    ipa_calc_crt = process_ipa_calc(creation_data)
    creation_stats = process_creation_stats(df)
    discomfort = process_discomfort_data(df)
    seq = process_seq_data(df)

    nav_trials = navigation_data.groupby(['modelID', 'methodID'])
    ipa_nav_trials = ipa_calc_nav.groupby(['modelID', 'methodID'])
    crt_trials = creation_data.groupby(['modelID', 'methodID'])
    ipa_crt_trials = ipa_calc_crt.groupby(['modelID', 'methodID'])
    discomfort_trials = discomfort.groupby(['ModelID', 'MethodID'])
    seq_trials = seq.groupby(['ModelID', 'MethodID'])

    nav_data = {}
    ipa_nav_data = {}
    crt_data = {}
    ipa_crt_data = {}
    discomfort_data = {}
    seq_crt = {}
    seq_nav = {}

    for i in range(4):
        for j in range(2,4):
            nav_data[(id, i, j)] = nav_trials.get_group((i,j)).mean()
            ipa_nav_data[(id, i, j)] = ipa_nav_trials.get_group((i,j)).mean()
            discomfort_data[(id, i, j)] = discomfort_trials.get_group((i,j)).mean()
            seq_crt[(id, i, j)] = seq_trials.get_group((i,j)).mean()
    
    for i in range(4):
        for j in range(0,2):
            crt_data[(id, i, j)] = crt_trials.get_group((i,j)).mean()
            ipa_crt_data[(id, i, j)] = ipa_crt_trials.get_group((i,j)).mean()
            seq_nav[(id, i, j)] = seq_trials.get_group((i,j)).mean()
    
    nav_index = pd.MultiIndex.from_product([[id], model_cats.categories, method_cats.categories[0:2]], names=data_names)
    crt_index = pd.MultiIndex.from_product([[id], model_cats.categories, method_cats.categories[2:4]], names=data_names)

    nav_data = pd.concat(nav_data, axis=1, names=data_names).T
    nav_data.index = nav_index
    nav_data.drop(columns=['time', 'modelID', 'methodID'], inplace=True)

    crt_data = pd.concat(crt_data, axis=1, names=data_names).T
    crt_data.index = crt_index
    crt_data.drop(columns=['time', 'modelID', 'methodID'], inplace=True)

    creation_stats.index = crt_index

    ipa_nav_data = pd.concat(ipa_nav_data, axis=1, names=data_names).T
    ipa_nav_data.index = nav_index
    ipa_nav_data.drop(columns=['modelID', 'methodID'], inplace=True)

    ipa_crt_data = pd.concat(ipa_crt_data, axis=1, names=data_names).T
    ipa_crt_data.index = crt_index
    ipa_crt_data.drop(columns=['modelID', 'methodID'], inplace=True)

    discomfort_data = pd.concat(discomfort_data, axis=1, names=data_names).T
    discomfort_data.index = nav_index
    discomfort_data['discomfort'] = discomfort_data['Value']
    discomfort_data.drop(columns=['time', 'ModelID', 'MethodID', 'Value'], inplace=True)

    seq_nav = pd.concat(seq_nav, axis=1, names=data_names).T
    seq_nav.index = nav_index
    seq_nav['seq'] = seq_nav['Value']
    seq_nav.drop(columns=['time', 'ModelID', 'MethodID', 'Value'], inplace=True)

    seq_crt = pd.concat(seq_crt, axis=1, names=data_names).T
    seq_crt.index = crt_index
    seq_crt['seq'] = seq_crt['Value']
    seq_crt.drop(columns=['time', 'ModelID', 'MethodID', 'Value'], inplace=True)

    df_crt = pd.concat([crt_data, ipa_crt_data, seq_crt, creation_stats], axis=0).stack().unstack()
    df_crt['PointPlaced'] = df_crt['PointPlaced'].astype(int).fillna(0)
    df_crt['Move'] = df_crt['Move'].astype(int).fillna(0)
    df_crt['PointDeleted'] = df_crt['PointDeleted'].astype(int).fillna(0)
    df_crt['Draw'] = df_crt['Draw'].astype(int).fillna(0)
    df_crt['Erase'] = df_crt['Erase'].astype(int).fillna(0)

    df_nav = pd.concat([nav_data, ipa_nav_data, discomfort_data, seq_nav], axis=0).stack().unstack()

    user_dfs_nav.append(df_nav)
    user_dfs_crt.append(df_crt)



In [21]:
user_data_nav = pd.concat(user_dfs_nav)
nav_index = pd.MultiIndex.from_product([ids, model_cats.categories, method_cats.categories[0:2]], names=data_names)
user_data_nav.index = nav_index
nav_dtype = {'luminance': 'float64', 'pupilDiameter': 'float64', 'pupil_lum_base': 'float64', 'adj_pupil': 'float64', 'IPA': 'float64', 'discomfort': 'int32', 'seq': 'int32'}
user_data_nav = user_data_nav.astype(nav_dtype)


user_data_crt = pd.concat(user_dfs_crt)
crt_index = pd.MultiIndex.from_product([ids, model_cats.categories, method_cats.categories[2:4]], names=data_names)
user_data_crt.index = crt_index
crt_dtype = {'luminance' : 'float64', 'pupilDiameter' : 'float64', 'pupil_lum_base' : 'float64', 'adj_pupil' : 'float64', 'IPA' : 'float64', 'seq' : 'int32', 'PointPlaced' : 'int32', 'Move' : 'int32', 'Draw' : 'int32', 'Erase' : 'int32', 'PointDeleted' : 'int32'}
user_data_crt = user_data_crt.astype(crt_dtype)

In [22]:
# nav_type_schema = {'luminance': 'numeric', 'pupilDiameter': 'numeric', 'pupil_lum_base': 'numeric', 'adj_pupil': 'numeric', 'IPA': 'numeric', 'discomfort': 'categorical', 'seq': 'categorical'}
# crt_type_schema = {'luminance': 'numeric', 'pupilDiameter': 'numeric', 'pupil_lum_base': 'numeric', 'adj_pupil': 'numeric', 'IPA': 'numeric', 'seq': 'categorical', 'PointPlaced': 'numeric', 'Move': 'numeric', 'Draw': 'numeric', 'Erase': 'numeric', 'PointDeleted': 'numeric'}
# profile_nav = ProfileReport(user_data_nav, title='User Data Report', correlations={"auto": {"calculate": True},}, type_schema=nav_type_schema)
# profile_crt = ProfileReport(user_data_crt, title='User Data Report', correlations={"auto": {"calculate": True},}, type_schema=crt_type_schema)

In [23]:
user_ids = []
user_nav_data = []
user_crt_data = []
user_models_nav = []
user_methods_nav = []
user_params = []
user_calibration = []
user_models_crt = []
user_methods_crt = []
user_seq = []
user_discomfort = []
user_ipa_nav = []
user_ipa_crt = []
user_pupil_data = []
user_creation_stats = []

user_dfs = []

for df in dfs:
    id = df['ExperimentStream']['UserID'][0]
    user_ids.append(id) 

    # Remove final empty row from survey data
    df['SurveyStream'] = df['SurveyStream'].replace(r'^\s*$', np.nan, regex=True).dropna()
    df['CreationStream'] = df['CreationStream'].replace(r'^\s*$', np.nan, regex=True).dropna()

    df['SurveyStream']['ModelID'] = df['SurveyStream']['ModelID'].astype(float)
    df['SurveyStream']['MethodID'] = df['SurveyStream']['MethodID'].astype(float)
    df['SurveyStream']['Value'] = df['SurveyStream']['Value'].astype(float)

    df['CreationStream']['ModelID'] = df['CreationStream']['ModelID'].astype(float)
    df['CreationStream']['MethodID'] = df['CreationStream']['MethodID'].astype(float)
    df['CreationStream']['EventType'] = df['CreationStream']['EventType'].astype("category")

    
    pupil_lum_df = process_gaze_luminance_data(df)
    calibration_data = process_calibration_data(pupil_lum_df, df)

    user_calibration.append(calibration_data)

    # Fit pupil response to luminance
    x_data = calibration_data['luminance']
    y_data = calibration_data['pupilDiameter']
    exp_mod = Model(pupil_func)
    params = exp_mod.make_params(a=1, b=4, c=0)
    result = exp_mod.fit(y_data, params, x=x_data)
    a = result.params['a'].value
    b = result.params['b'].value
    c = result.params['c'].value

    user_params.append(pd.DataFrame({'params': [a, b, c]}, index=['a', 'b', 'c']))

    pupil_data = pupil_lum_df.loc[(pupil_lum_df['methodID'] < 5), ['time', 'luminance', 'pupilDiameter']]
    pupil_data.reset_index(drop=True, inplace=True)
    pupil_data['time'] = pupil_data['time'] - pupil_data['time'][0]
    pupil_data['pupil_lum_base'] = pupil_func(pupil_data['luminance'], a, b, c)
    pupil_data['adj_pupil'] = pupil_data['pupilDiameter'] - pupil_data['pupil_lum_base']

    user_pupil_data.append(pupil_data)

    navigation_data = process_navigation_data(pupil_lum_df, df, a, b, c)

    user_nav_data.append(navigation_data)

    ipa_calc = process_ipa_calc(navigation_data)
    ipa_avg = ipa_calc.groupby(['methodID']).mean()
    
    user_ipa_nav.append(ipa_avg)

    navigation_avg = navigation_data.groupby(level=0).mean()

    models = navigation_avg.reset_index(drop=True)
    model_avg = models.groupby(['modelID']).mean()
    model_avg.drop(columns=['methodID'], inplace=True)
    methods = navigation_avg.reset_index(drop=True)
    method_avg = methods.groupby(['methodID']).mean()
    method_avg.drop(columns=['modelID'], inplace=True)

    user_models_nav.append(model_avg)
    user_methods_nav.append(method_avg)

    creation_data = process_creation_data(pupil_lum_df, df, a, b, c)

    user_crt_data.append(creation_data)

    ipa_calc = process_ipa_calc(creation_data)
    ipa_avg = ipa_calc.groupby(['methodID']).mean()

    user_ipa_crt.append(ipa_avg)

    creation_stats = process_creation_stats(df)
    method_avg = creation_stats.groupby(['MethodID']).mean()

    user_creation_stats.append(method_avg)

    creation_avg = creation_data.groupby(level=0).mean()

    models = creation_avg.reset_index(drop=True)
    model_avg = models.groupby(['modelID']).mean()
    model_avg.drop(columns=['methodID'], inplace=True)
    methods = creation_avg.reset_index(drop=True)
    method_avg = methods.groupby(['methodID']).mean()
    method_avg.drop(columns=['modelID'], inplace=True)

    user_models_crt.append(model_avg)
    user_methods_crt.append(method_avg)
    
    user_discomfort.append(process_discomfort_data(df))
    user_seq.append(process_seq_data(df))

params = pd.concat(user_params, keys=user_ids, names=['UserID'])
model_data_nav = pd.concat(user_models_nav, keys=user_ids, names=['UserID'])
method_data_nav = pd.concat(user_methods_nav, keys=user_ids, names=['UserID'])
model_data_crt = pd.concat(user_models_crt, keys=user_ids, names=['UserID'])
method_data_crt = pd.concat(user_methods_crt, keys=user_ids, names=['UserID'])
nav_data = pd.concat(user_nav_data, keys=user_ids, names=['UserID'])
crt_data = pd.concat(user_crt_data, keys=user_ids, names=['UserID'])
seq_data = pd.concat(user_seq, keys=user_ids, names=['UserID'])
discomfort_data = pd.concat(user_discomfort, keys=user_ids, names=['UserID'])
calibration_data = pd.concat(user_calibration, keys=user_ids, names=['UserID'])
ipa_data_nav = pd.concat(user_ipa_nav, keys=user_ids, names=['UserID'])
ipa_data_crt = pd.concat(user_ipa_crt, keys=user_ids, names=['UserID'])
pupil_data = pd.concat(user_pupil_data, keys=user_ids, names=['UserID'])
creation_stats = pd.concat(user_creation_stats, keys=user_ids, names=['UserID'])

### Draw Traces for all Eye Diameter Data

Uncomment to view the traces of the eye diameter data for all participants.

In [24]:
#draw traces for each user
# fig = go.Figure()
# pupil_data.dropna(inplace=True)
# for user in user_ids:
#     fig.add_trace(go.Scatter(x=pupil_data.loc[user]['time'].dt.total_seconds(), y=pupil_data.loc[user]['pupilDiameter'], mode='lines', name=user))
# fig.update_layout(title='Pupil Diameter for each User', xaxis_title='Time', yaxis_title='Pupil Diameter (mm)')
# fig.update_xaxes(range=[60,240])
# fig.show()


## Statistical Analysis

Method 0: One-handed drawing interface (path creation)

Method 1: Two-handed point-placement interface (path creation)

Method 2: 4DOF locomotion (navigation)

Method 3: 6DOF locomotion (navigation)

### Navigation Workload

#### IPA Evaluation

In [228]:
ipa_nav = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_nav = ipa_nav.unstack(level=(2))

outliers_4dof = iqr_outlier_indices(ipa_nav['4DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_4dof].index)

outliers_6dof = iqr_outlier_indices(ipa_nav['6DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_6dof].index)

ipa_4dof = ipa_nav['4DoF']
ipa_6dof = ipa_nav['6DoF']

#Test for normality
n_stat, n_p = stats.shapiro(ipa_nav)

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(ipa_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(ipa_6dof)
    header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [ipa_4dof.median(), ipa_6dof.median()], [iqr_4dof, iqr_6dof], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]
    stat, p = stats.wilcoxon(ipa_4dof, ipa_6dof)
    prefix=['W = ','']

else:
    header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    values = [['<b>4-DoF</b>', '<b>6-DoF</b'], [ipa_4dof.mean(), ipa_6dof.mean()], [ipa_4dof.std(), ipa_6dof.std()], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]
    stat, p = stats.ttest_rel(ipa_4dof, ipa_6dof)
    prefix=['t = ','']

In [222]:
stats_table = go.Table(columnwidth=[5,4,4,4,4], name='Summary')
stats_table.header = dict(
    values=header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

iqr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

iqr_fig.add_trace(stats_table, row=2, col=1)
iqr_fig.add_trace(go.Box(y=ipa_nav['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Box(y=ipa_nav['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Table(
        header=dict(values=["<b>Result</b>", "<b>p-value</b>"], align="center",),
        cells=dict(values=[stat, p], align = "center", format=["0.1f", ".6f"], prefix=prefix)),
        row=1, col=1)

iqr_fig.update_layout(
    title_text="IPA by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

iqr_fig.update_yaxes(range=[0,max(1.1*ipa_4dof.max(), 1.1*ipa_6dof.max())])
iqr_fig.show()

#### TEPR Evaluation

In [27]:
tepr = user_data_nav.loc[(slice(None), slice(None),  slice(None)), 'adj_pupil']
tepr_4dof = tepr.loc[(slice(None), slice(None), '4DoF')]
tepr_6dof = tepr.loc[(slice(None), slice(None), '6DoF')]

statt, pt = stats.shapiro(tepr)

#wilcoxon test
stat, p = stats.wilcoxon(tepr_4dof, tepr_6dof)

#paired t-test
t_stat, p_val = stats.ttest_rel(tepr_4dof, tepr_6dof)

#descriptive stats (avg, median, std)
print('Method 2')
print('Mean = %.3f' % tepr_4dof.mean())
print('Median = %.3f' % tepr_4dof.median())
print('Std = %.3f' % tepr_4dof.std())

print('Method 3')
print('Mean = %.3f' % tepr_6dof.mean())
print('Median = %.3f' % tepr_6dof.median())
print('Std = %.3f' % tepr_6dof.std())

print('Shapiro-Wilk T = %.3f, p = %.3f' % (statt, pt))
print('Wilcoxon = %.3f, p = %.3f' % (stat, p))
print('Paired t-test = %.3f, p = %.3f' % (t_stat, p_val))

Method 2
Mean = 0.512
Median = 0.491
Std = 0.302
Method 3
Mean = 0.520
Median = 0.504
Std = 0.287
Shapiro-Wilk T = 0.968, p = 0.000
Wilcoxon = 2032.000, p = 0.279
Paired t-test = -0.336, p = 0.738


In [28]:
df = tepr.reset_index(level=[0,1,2])
nav_tepr_fig = px.box(df, x='method', y ='adj_pupil', color='method', notched=True, title='TEPR by Navigation Method', color_discrete_sequence=px.colors.qualitative.D3)
nav_tepr_fig.update_yaxes(range=[-0.1,1.25])
nav_tepr_fig.update_layout(
    xaxis_title='Method',
    yaxis_title='TEPR',
    width=600,
)
nav_tepr_fig.show()

c:\Users\Courtney\anaconda3\envs\path_data\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



### Creation Workload by Method

#### IPA Evaluation

In [194]:
ipa_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_crt = ipa_crt.unstack(level=(2))
uni_outliers = iqr_outlier_indices(ipa_crt['unimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[uni_outliers].index)
bi_outliers = iqr_outlier_indices(ipa_crt['bimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[bi_outliers].index)
ipa_uni = ipa_crt['unimanual']
ipa_bi = ipa_crt['bimanual']

#Test for normality
stat, p = stats.shapiro(ipa_crt)

uni_iqr, uni_q1, uni_q3 = iqr_stats(ipa_uni)
bi_iqr, bi_q1, bi_q3 = iqr_stats(ipa_bi)

if p < 0.05:
    header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], [ipa_uni.median(), ipa_bi.median()], [uni_iqr, bi_iqr], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]
    stat, p = stats.wilcoxon(ipa_uni, ipa_bi)
    prefix=['W = ','']

else:
    header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    values = [['<b>Unimanual</b>', '<b>Bimanual</b'], [ipa_uni.mean(), ipa_bi.mean()], [ipa_uni.std(), ipa_bi.std()], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]
    stat, p = stats.ttest_rel(ipa_uni, ipa_bi)
    prefix=['t = ','']


In [195]:
stats_table = go.Table(columnwidth=[5,4,4,4,4], name='Summary')
stats_table.header = dict(
    values=header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

iqr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

iqr_fig.add_trace(stats_table, row=2, col=1)
iqr_fig.add_trace(go.Box(y=ipa_crt['bimanual'], name="Bimanual", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Box(y=ipa_crt['unimanual'], name="Unimanual", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Table(
        header=dict(values=["<b>Result</b>", "<b>p-value</b>"], align="center",),
        cells=dict(values=[stat, p], align = "center", format=["0.1f", ".6f"], prefix=prefix)),
        row=1, col=1)

iqr_fig.update_layout(
    title_text="IPA by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

iqr_fig.update_yaxes(range=[0,max(1.1*ipa_uni.max(), 1.1*ipa_bi.max())])
iqr_fig.show()

#### TEPR Evaluation

In [31]:
#descriptive stats (avg, median, std)
print('Method 0')
print('Mean = %.3f' % method_tepr_0.mean())
print('Median = %.3f' % method_tepr_0.median())
print('Std = %.3f' % method_tepr_0.std())

print('Method 1')
print('Mean = %.3f' % method_tepr_1.mean())
print('Median = %.3f' % method_tepr_1.median())
print('Std = %.3f' % method_tepr_1.std())

#shapiro-wilk test
stat, p = stats.shapiro(method_crt_tepr)
print('Shapiro-Wilk total = %.3f, p = %.5f' % (stat, p))

#wilcoxon test
stat, p = stats.wilcoxon(method_tepr_0, method_tepr_1)
print('Wilcoxon = %.3f, p = %.5f' % (stat, p))

#paired t-test
t_stat, p_val = stats.ttest_rel(method_tepr_0, method_tepr_1)
print('Paired t-test = %.3f, p = %.5f' % (t_stat, p_val))


Method 0


NameError: name 'method_tepr_0' is not defined

In [ ]:
df = method_data_crt.loc[(slice(None), slice(None)), 'adj_pupil'].reset_index('methodID')
crt_tepr_fig = px.box(df, x='methodID', y ='adj_pupil', color='methodID', notched=True, title='TEPR by Creation Method', color_discrete_sequence=px.colors.qualitative.D3)
crt_tepr_fig.update_yaxes(range=[0,1.5])
crt_tepr_fig.update_layout(
    xaxis_title='Method',
    yaxis_title='TEPR',
    width=600,
)
crt_tepr_fig.show()


In [ ]:
model_crt_tepr = model_data_crt.loc[(slice(None), slice(None)), ('pupilDiameter', 'pupil_lum_base', 'adj_pupil')]

model_tepr_0 = model_crt_tepr.loc[(slice(None), 0), 'adj_pupil']
model_tepr_1 = model_crt_tepr.loc[(slice(None), 1), 'adj_pupil']
model_tepr_2 = model_crt_tepr.loc[(slice(None), 2), 'adj_pupil']
model_tepr_3 = model_crt_tepr.loc[(slice(None), 3), 'adj_pupil']

#shapiro-wilk test
stat, p = stats.shapiro(model_crt_tepr['adj_pupil'])
print('Shapiro-Wilk total = %.3f, p = %.3f' % (stat, p))

#Friedman test
stat, p = stats.friedmanchisquare(model_tepr_0, model_tepr_1, model_tepr_2, model_tepr_3)
print('Friedman = %.3f, p = %.6f' % (stat, p))

Shapiro-Wilk total = 0.968, p = 0.019
Friedman = 12.550, p = 0.005718


In [ ]:
creation_stats = creation_stats.fillna(0)
creation_stats.index = creation_stats.index.set_levels(creation_stats.index.levels[1].astype('category'), level=1)
stats_avg = creation_stats.groupby('MethodID').mean()
stats_avg.index = stats_avg.index.astype('string')
fig = px.bar(stats_avg)
fig.show()

C:\Users\Courtney\AppData\Local\Temp\ipykernel_23824\3020217953.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Discomfort Scores

In [ ]:
discomfort_method = discomfort_data.loc[(slice(None), slice(None)), ['Value', 'MethodID', 'ModelID']]
discomfort_method['Value'] = discomfort_method['Value'].astype(float)
discomfort_method['MethodID'] = discomfort_method['MethodID'].astype(float)
discomfort_method['ModelID'] = discomfort_method['ModelID'].astype(float)

# find the average for each user for each method
discomfort_avg = discomfort_method.groupby(['UserID', 'MethodID']).mean()
discomfort_method_2 = discomfort_avg.loc[(slice(None), 2), 'Value']
discomfort_method_3 = discomfort_avg.loc[(slice(None), 3), 'Value']

print('Method 2')
print('Mean = %.3f' % discomfort_method_2.mean())
print('Median = %.3f' % discomfort_method_2.median())
print('Std = %.3f' % discomfort_method_2.std())

print('Method 3')
print('Mean = %.3f' % discomfort_method_3.mean())
print('Median = %.3f' % discomfort_method_3.median())
print('Std = %.3f' % discomfort_method_3.std())

#shapiro-wilk test
stat, p = stats.shapiro(discomfort_avg['Value'])
print('Shapiro-Wilk total = %.3f, p = %.5f' % (stat, p))

#wilcoxon test
stat, p = stats.wilcoxon(discomfort_method_2, discomfort_method_3, zero_method='pratt')
print('Wilcoxon = %.3f, p = %.5f' % (stat, p))

#paired t-test
t_stat, p_val = stats.ttest_rel(discomfort_method_2, discomfort_method_3)
print('Paired t-test = %.3f, p = %.5f' % (t_stat, p_val))




Method 2
Mean = 2.075
Median = 0.900
Std = 2.172
Method 3
Mean = 2.408
Median = 2.000
Std = 2.326
Shapiro-Wilk total = 0.859, p = 0.00004
Wilcoxon = 86.500, p = 0.09353
Paired t-test = -1.718, p = 0.09918


c:\Users\Courtney\anaconda3\envs\data_profiler\Lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.



In [ ]:
discomfort_fig = px.box(discomfort_method, x='MethodID', y='Value', color='MethodID', title='Discomfort Ratings by Navigation Method', notched=True, color_discrete_sequence=px.colors.qualitative.D3)
discomfort_fig.update_yaxes(range=[0,10])
discomfort_fig.update_layout(
    xaxis_title='Method',
    yaxis_title='Discomfort Rating',
    width=600,
)
discomfort_fig.show()

In [ ]:
model_discomfort = discomfort_method.groupby(['UserID', 'ModelID']).mean()

discomfort_model_0 = model_discomfort.loc[(slice(None), 0), 'Value']
discomfort_model_1 = model_discomfort.loc[(slice(None), 1), 'Value']
discomfort_model_2 = model_discomfort.loc[(slice(None), 2), 'Value']
discomfort_model_3 = model_discomfort.loc[(slice(None), 3), 'Value']

#shapiro-wilk test
stat, p = stats.shapiro(model_discomfort['Value'])
print('Shapiro-Wilk total = %.3f, p = %.6f' % (stat, p))

#Friedman test
stat, p = stats.friedmanchisquare(discomfort_model_0, discomfort_model_1, discomfort_model_2, discomfort_model_3)
print('Friedman = %.3f, p = %.6f' % (stat, p))

Shapiro-Wilk total = 0.865, p = 0.000000
Friedman = 1.435, p = 0.697402


In [ ]:
seq_method = seq_data.loc[(slice(None), slice(None)), ['Value', 'MethodID', 'ModelID']]
seq_method['Value'] = seq_method['Value'].astype(float)
seq_method['MethodID'] = seq_method['MethodID'].astype(float)
seq_method['ModelID'] = seq_method['ModelID'].astype(float)

# find the average for each user for each method
seq_avg = seq_method.groupby(['UserID', 'MethodID']).mean()
seq_method_0 = seq_avg.loc[(slice(None), 0), 'Value']
seq_method_1 = seq_avg.loc[(slice(None), 1), 'Value']
seq_method_2 = seq_avg.loc[(slice(None), 2), 'Value']
seq_method_3 = seq_avg.loc[(slice(None), 3), 'Value']

#shapiro-wilk test
statt01, pt01 = stats.shapiro(seq_avg.loc[(slice(None), [0,1]), 'Value'])
statt23, pt23 = stats.shapiro(seq_avg.loc[(slice(None), [2,3]), 'Value'])

#wilcoxon test
statw_01, p_w_01 = stats.wilcoxon(seq_method_0, seq_method_1, zero_method='pratt')
statw_23, p_w_23 = stats.wilcoxon(seq_method_2, seq_method_3, zero_method='pratt')

#paired t-test
t_stat, p_val = stats.ttest_rel(seq_method_0, seq_method_1)
t_stat2, p_val2 = stats.ttest_rel(seq_method_2, seq_method_3)

#descriptive stats (avg, median, std)
print('Method 0')
print('Mean = %.3f' % seq_method_0.mean())
print('Median = %.3f' % seq_method_0.median())
print('Std = %.3f' % seq_method_0.std())

print('Method 1')
print('Mean = %.3f' % seq_method_1.mean())
print('Median = %.3f' % seq_method_1.median())
print('Std = %.3f' % seq_method_1.std())

print('Method 2')
print('Mean = %.3f' % seq_method_2.mean())
print('Median = %.3f' % seq_method_2.median())
print('Std = %.3f' % seq_method_2.std())

print('Method 3')
print('Mean = %.3f' % seq_method_3.mean())
print('Median = %.3f' % seq_method_3.median())
print('Std = %.3f' % seq_method_3.std())

print('Shapiro-Wilk 01 = %.3f, p = %.5f' % (statt01, pt01))
print('Shapiro-Wilk 23 = %.3f, p = %.5f' % (statt23, pt23))

print('Wilcoxon 01 = %.3f, p = %.5f' % (statw_01, p_w_01))
print('Wilcoxon 23 = %.3f, p = %.5f' % (statw_23, p_w_23))

Method 0
Mean = 0.760
Median = 0.750
Std = 0.735
Method 1
Mean = 1.010
Median = 0.875
Std = 1.017
Method 2
Mean = 0.406
Median = 0.000
Std = 0.570
Method 3
Mean = 1.010
Median = 1.125
Std = 0.868
Shapiro-Wilk 01 = 0.874, p = 0.00010
Shapiro-Wilk 23 = 0.837, p = 0.00001
Wilcoxon 01 = 80.500, p = 0.21090
Wilcoxon 23 = 49.000, p = 0.00701


c:\Users\Courtney\anaconda3\envs\data_profiler\Lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.



In [ ]:
seq_fig = px.box(seq_method, x='MethodID', y='Value', color='MethodID', title='SEQ Ratings by Method', notched=True, color_discrete_sequence=px.colors.qualitative.D3)
seq_fig.update_yaxes(range=[0,6])
seq_fig.update_layout(
    xaxis_title='Method',
    yaxis_title='SEQ Rating',
    width=600,
)
seq_fig.show()